In [29]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pandas as pd
import nltk
from textblob import TextBlob

In [3]:
table_business = pd.read_parquet("../Data/data_procesada/Dataset_Business.parquet")
table_business

,id_business,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Doctors, Traditional Chinese Medicine, Naturop...",None
1,3,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
2,4,Perkiomen Valley Brewery,101 Walnut St,Green Lane,Pennsylvania,18054,40.338183,-75.471659,4.5,13,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Brewpubs, Breweries, Food","{'Friday': '12:0-22:0', 'Monday': None, 'Satur..."
3,7,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,Florida,33707,27.766590,-82.732983,3.5,5,1,None,"Synagogues, Religious Organizations","{'Friday': '9:0-17:0', 'Monday': '9:0-17:0', '..."
4,10,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,Florida,34639,28.190459,-82.457380,3.5,6,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Department Stores, Shopping, Fashion","{'Friday': '9:30-21:30', 'Monday': '9:30-21:30..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81048,150334,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,Pennsylvania,19103,39.945966,-75.169666,4.5,39,1,"{'AcceptsInsurance': 'None', 'AgesAllowed': No...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Friday': '7:0-20:0', 'Monday': '7:0-20:0', '..."
81049,150336,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,Pennsylvania,19147,39.935982,-75.158665,4.5,35,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Mexican","{'Friday': '11:0-2:0', 'Monday': '11:0-22:0', ..."
81050,150337,The Vac & Sew Center,"200 Haddonfield Berlin Rd, Ste 5",Voorhees,New Jersey,8043,39.857700,-74.987230,4.0,5,0,None,"Appliances & Repair, Home & Garden, Appliances...","{'Friday': '9:30-18:0', 'Monday': '9:30-18:0',..."
81051,150339,The Plum Pit,4405 Pennell Rd,Aston,Pennsylvania,19014,39.856185,-75.427725,4.5,14,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa..."


In [5]:
null_indices_hora = table_business['hours'].isnull()
null_indices_hora

0         True
1        False
2        False
3        False
4        False
         ...  
81048    False
81049    False
81050    False
81051    False
81052    False
Name: hours, Length: 81053, dtype: bool

In [6]:
random_row = table_business.sample()
random_hours_value = random_row['hours'].values[0]
print(random_hours_value)

None


In [7]:
horario_general = {'Friday': '9:0-20:0', 'Monday': '9:0-20:0', 'Saturday': '8:30-15:30', 'Sunday': None, 'Thursday': '9:0-20:0', 'Tuesday': '9:0-20:0', 'Wednesday': '9:0-20:0'}

In [8]:
table_business.loc[null_indices_hora, 'hours'] = table_business.loc[null_indices_hora].apply(lambda row: horario_general, axis=1)

In [10]:
null_indices = table_business['hours'].isnull()
null_indices


0        False
1        False
2        False
3        False
4        False
         ...  
81048    False
81049    False
81050    False
81051    False
81052    False
Name: hours, Length: 81053, dtype: bool

In [11]:
import geocoder

def get_postal_code(latitude, longitude):
    location = geocoder.osm([latitude, longitude], method='reverse')
    return location.postal



In [12]:
#reemplazo los valores nulos en la tabla
table_business.loc[null_indices, 'postal_code'] = table_business.loc[null_indices].apply(lambda row: get_postal_code(row['latitude'], row['longitude']), axis=1)


In [13]:

valores_nulos = table_business[table_business['postal_code'].isnull()]
valores_nulos
#este valor nulo me manda al medio del mar por eso no tiene postal code. Considerar si lo borramos o no

,id_business,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours


In [14]:
faltante = get_postal_code(32.105261,110.942001)
print(faltante)

None


In [15]:
table_business['postal_code'] = table_business['postal_code'].astype(float)
#REVISAR MAÑANA

In [16]:
duplicates_mask = table_business['address'].duplicated(keep=False)
duplicated_addresses = table_business[duplicates_mask]
duplicated_addresses

,id_business,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
5,11,Vietnamese Food Truck,,Tampa Bay,Florida,33602.0,27.955269,-82.456320,4.0,10,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Vietnamese, Food, Restaurants, Food Trucks","{'Friday': '11:0-14:0', 'Monday': '11:0-14:0',..."
7,14,Zio's Italian Market,2575 E Bay Dr,Largo,Florida,33771.0,27.916116,-82.760461,4.5,100,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Food, Delis, Italian, Bakeries, Restaurants","{'Friday': '10:0-20:0', 'Monday': '10:0-18:0',..."
8,15,Tuna Bar,205 Race St,Philadelphia,Pennsylvania,19106.0,39.953949,-75.143226,4.0,245,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Sushi Bars, Restaurants, Japanese","{'Friday': '13:30-23:0', 'Monday': None, 'Satu..."
13,28,Bar One,767 S 9th St,Philadelphia,Pennsylvania,19147.0,39.939825,-75.157447,4.0,65,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Friday': '12:0-2:0', 'Monday': '16:0-0:0', '..."
14,31,DeSandro on Main,4105 Main St,Philadelphia,Pennsylvania,19127.0,40.022466,-75.218314,3.0,41,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Pizza, Restaurants, Salad, Soup","{'Friday': '17:0-0:30', 'Monday': None, 'Satur..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81042,150319,Kitchen Gia,3716 Spruce St,Philadelphia,Pennsylvania,19104.0,39.951018,-75.198240,3.0,22,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Coffee & Tea, Food, Sandwiches, American (Trad...","{'Friday': '9:0-17:0', 'Monday': '9:0-19:30', ..."
81043,150320,Intermix,1718 Walnut St,Philadelphia,Pennsylvania,19103.0,39.950060,-75.170129,2.0,10,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Fashion, Shopping, Women's Clothing","{'Friday': '10:0-19:0', 'Monday': '10:0-19:0',..."
81045,150326,2601 Parkway Apartments,2601 Pennsylvania Ave,Philadelphia,Pennsylvania,19130.0,39.968806,-75.180608,3.0,6,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Real Estate, Apartments, Home Services","{'Friday': '9:0-20:0', 'Monday': '9:0-20:0', '..."
81049,150336,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,Pennsylvania,19147.0,39.935982,-75.158665,4.5,35,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Mexican","{'Friday': '11:0-2:0', 'Monday': '11:0-22:0', ..."


In [17]:
table_business.drop_duplicates(subset=['latitude', 'longitude', 'postal_code'], inplace=True)
table_business

,id_business,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101.0,34.426679,-119.711197,5.0,7,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Doctors, Traditional Chinese Medicine, Naturop...","{'Friday': '9:0-20:0', 'Monday': '9:0-20:0', '..."
1,3,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107.0,39.955505,-75.155564,4.0,80,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
2,4,Perkiomen Valley Brewery,101 Walnut St,Green Lane,Pennsylvania,18054.0,40.338183,-75.471659,4.5,13,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Brewpubs, Breweries, Food","{'Friday': '12:0-22:0', 'Monday': None, 'Satur..."
3,7,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,Florida,33707.0,27.766590,-82.732983,3.5,5,1,None,"Synagogues, Religious Organizations","{'Friday': '9:0-17:0', 'Monday': '9:0-17:0', '..."
4,10,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,Florida,34639.0,28.190459,-82.457380,3.5,6,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Department Stores, Shopping, Fashion","{'Friday': '9:30-21:30', 'Monday': '9:30-21:30..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81046,150329,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24",Philadelphia,Pennsylvania,19147.0,39.932756,-75.144504,3.0,32,1,"{'AcceptsInsurance': 'False', 'AgesAllowed': N...","Day Spas, Beauty & Spas, Skin Care, Massage","{'Friday': '9:0-22:0', 'Monday': '9:0-22:0', '..."
81048,150334,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,Pennsylvania,19103.0,39.945966,-75.169666,4.5,39,1,"{'AcceptsInsurance': 'None', 'AgesAllowed': No...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Friday': '7:0-20:0', 'Monday': '7:0-20:0', '..."
81050,150337,The Vac & Sew Center,"200 Haddonfield Berlin Rd, Ste 5",Voorhees,New Jersey,8043.0,39.857700,-74.987230,4.0,5,0,None,"Appliances & Repair, Home & Garden, Appliances...","{'Friday': '9:30-18:0', 'Monday': '9:30-18:0',..."
81051,150339,The Plum Pit,4405 Pennell Rd,Aston,Pennsylvania,19014.0,39.856185,-75.427725,4.5,14,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa..."


In [18]:
duplicates_mask = table_business.duplicated(subset=['latitude', 'longitude', 'postal_code'], keep=False)
num_duplicates = duplicates_mask.sum()
num_duplicates

0

In [19]:
rows = []

# Iterar sobre cada fila del DataFrame original
for index, row in table_business.iterrows():
    # Obtener las categorías de la fila actual
    categories = row['categories']
    
    # Si hay categorías
    if categories:
        # Dividir las categorías y eliminar espacios en blanco
        categories_list = [category.strip() for category in categories.split(',')]
        
        # Iterar sobre cada categoría y duplicar la fila actual
        for category in categories_list:
            # Crear una copia de la fila actual
            new_row = row.copy()
            # Asignar la categoría actual a la fila copiada
            new_row['categories'] = category
            # Agregar la fila copiada a la lista
            rows.append(new_row)

# Crear un nuevo DataFrame con las filas duplicadas
table_business = pd.DataFrame(rows)


In [23]:
table_business = table_business.drop(columns=['attributes', 'postal_code', 'latitude', 'longitude', 'address', 'hours'])

In [25]:
table_business

,id_business,name,city,state,stars,review_count,is_open,categories
0,0,"Abby Rappoport, LAC, CMQ",Santa Barbara,California,5.0,7,0,Doctors
0,0,"Abby Rappoport, LAC, CMQ",Santa Barbara,California,5.0,7,0,Traditional Chinese Medicine
0,0,"Abby Rappoport, LAC, CMQ",Santa Barbara,California,5.0,7,0,Naturopathic/Holistic
0,0,"Abby Rappoport, LAC, CMQ",Santa Barbara,California,5.0,7,0,Acupuncture
0,0,"Abby Rappoport, LAC, CMQ",Santa Barbara,California,5.0,7,0,Health & Medical
...,...,...,...,...,...,...,...,...
81051,150339,The Plum Pit,Aston,Pennsylvania,4.5,14,1,Event Planning & Services
81052,150345,Sic Ink,Apollo beach,Florida,4.5,9,1,Beauty & Spas
81052,150345,Sic Ink,Apollo beach,Florida,4.5,9,1,Permanent Makeup
81052,150345,Sic Ink,Apollo beach,Florida,4.5,9,1,Piercing


In [24]:
table_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328700 entries, 0 to 81052
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id_business   328700 non-null  int64  
 1   name          328700 non-null  object 
 2   city          328700 non-null  object 
 3   state         328700 non-null  object 
 4   stars         328700 non-null  float64
 5   review_count  328700 non-null  int64  
 6   is_open       328700 non-null  int64  
 7   categories    328700 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 22.6+ MB


In [27]:
df_reviews_yelp = pd.read_parquet('../Data/data_procesada/Dataset_Review.parquet')
df_reviews_yelp

,user_id,stars,useful,funny,cool,text,date,id_review,id_user,id_business
0,mh_-eMZ6K5RLWhZyISBhwA,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0,124234,1867
1,OyoGAe7OKpv6SyGZT5g77Q,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,1,51327,12886
2,_7bHUi9Uuf5__HHc_Q8guQ,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,3,164430,6316
3,eUta8W_HdHMXPzLBBZhL1A,1.0,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31,5,132438,2763
4,yfFzsLmaWF2d4Sr0UNbBgg,5.0,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,7,72094,3222
...,...,...,...,...,...,...,...,...,...,...
3776697,QIXYkyAbTvgePUOH0-cRFg,2.0,0,0,0,The front desk staff is very good and they hav...,2008-09-23 02:52:37,6990271,782062,147335
3776698,7L7GL5Pi2cf8mbm2Dpw4zw,5.0,1,0,1,It is very rare for a restaurant to be this go...,2022-01-17 22:36:01,6990272,46895,136634
3776699,RKPkxOYQlM0BjhM-H6_vAw,4.0,3,0,2,"Good, maybe very good. I went for lunch, so t...",2015-01-23 23:52:03,6990274,55742,139815
3776700,Zo0th2m8Ez4gLSbHftiQvg,5.0,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,6990276,1192958,144720


In [30]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
def analizar_sentimiento(texto):
    blob = TextBlob(texto)
    sentimiento = blob.sentiment.polarity
    return sentimiento

In [ ]:
df_reviews_yelp['sentimiento'] = df_reviews_yelp['text'].apply(analizar_sentimiento)

In [ ]:
analisis_por_estrellas = df_reviews_yelp.groupby(['id_business', 'stars'])['sentimiento'].mean().reset_index()

In [ ]:
def escalar_sentimiento(sentimiento):
    # Escalar el sentimiento de -1 a 1 a un rango de 1 a 10
    sentimiento_escalado = (sentimiento + 1) * 5
    return sentimiento_escalado

In [ ]:
df_reviews_yelp['sentimiento_escalado'] = df_reviews_yelp['sentimiento'].apply(escalar_sentimiento)

In [ ]:
analisis_por_estrellas = df_reviews_yelp.groupby(['id_business', 'stars'])['sentimiento_escalado'].mean().reset_index()

In [ ]:
# Calcular los valores medios de sentimiento y sentimiento_escalado para cada id_business
df_sentimiento_agrupado = df_reviews_yelp.groupby('id_business').agg({'sentimiento': 'mean', 'sentimiento_escalado': 'mean'}).reset_index()

# Fusionar los valores medios con el DataFrame df_business
df_business_agrupado = df_reviews_yelp.merge(df_sentimiento_agrupado, on='id_business', how='left')

# Eliminar duplicados después de la fusión
df_business_agrupado.drop_duplicates(subset=['id_business'], inplace=True)

In [ ]:
#df_business_agrupado.to_parquet("../Data/data_procesada/Data_business_ml.parquet")